# CSV to GraphB Migration

This notebook is a simple assigment of migrating data from a CSV file to a Graph Database using the GraphB library.

Done by: Abdulmunim Jundurahman


In [1]:
import pandas as pd 

# Let us first do EDA on the data before making decisions

In [90]:
file_path = "data/whc-sites-2018-small.csv" # found the data from https://github.com/csev/pg4e/blob/master/assn/pg4e_model/whc-sites-2018-small-orig.csv

df = pd.read_csv(file_path)

df.head()

,name,description,justification,year,longitude,latitude,area_hectares,category,state,region,iso
0,Cultural Landscape and Archaeological Remains ...,<p>The cultural landscape and archaeological r...,<p><em>Criterion (i):</em> The Buddha statues ...,2003,67.825250,34.846940,158.9265,Cultural,Afghanistan,Asia and the Pacific,af
1,Minaret and Archaeological Remains of Jam,"<p>The 65m-tall Minaret of Jam is a graceful, ...",<p><em>Criterion (ii):</em> The innovative arc...,2002,64.515889,34.396417,70.0000,Cultural,Afghanistan,Asia and the Pacific,af
2,Historic Centres of Berat and Gjirokastra,<p>Berat and Gjirokastra are inscribed as rare...,NaN,2005,20.133333,40.069444,58.9000,Cultural,Albania,Europe and North America,al
3,Butrint,"<p>Inhabited since prehistoric times, Butrint ...",NaN,1992,20.026111,39.751111,NaN,Cultural,Albania,Europe and North America,al
4,Al Qal'a of Beni Hammad,<p>In a mountainous site of extraordinary beau...,NaN,1980,4.786840,35.818440,150.0000,Cultural,Algeria,Arab States,dz


In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           1044 non-null   object 
 1   description    1044 non-null   object 
 2   justification  321 non-null    object 
 3   year           1044 non-null   int64  
 4   longitude      1044 non-null   float64
 5   latitude       1044 non-null   float64
 6   area_hectares  1033 non-null   float64
 7   category       1044 non-null   object 
 8   state          1044 non-null   object 
 9   region         1044 non-null   object 
 10  iso            1043 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 89.8+ KB


In [92]:
# Let us clean the data, escape double quotes and convert the data types

df['description'] = df['description'].apply(lambda x: x.replace('"', '\\"') if isinstance(x, str) else x)
df['justification'] = df['justification'].apply(lambda x: x.replace('"', '\\"') if isinstance(x, str) else x)
df['year'] = df['year'].astype(int)
df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)
df['area_hectares'] = df['area_hectares'].apply(lambda x: 0 if pd.isna(x) else float(x))

In [93]:
# categories

unique_categories = df['category'].unique()
print("Unique Categories:", unique_categories)

# states

unique_states = df['state'].unique()
print("Unique States:", unique_states)

# regions

unique_regions = df['region'].unique()
print("Unique Regions:", unique_regions)

Unique Categories: ['Cultural' 'Mixed' 'Natural']
Unique States: ['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus' 'Belgium'
 'Belize' 'Benin' 'Bolivia (Plurinational State of)'
 'Bosnia and Herzegovina' 'Botswana' 'Brazil' 'Bulgaria' 'Burkina Faso'
 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada' 'Central African Republic'
 'Chad' 'Chile' 'China' 'Colombia' 'Costa Rica' "C te d'Ivoire" 'Croatia'
 'Cuba' 'Cyprus' 'Czechia' "Democratic People's Republic of Korea"
 'Democratic Republic of the Congo' 'Denmark' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Eritrea' 'Estonia'
 'Ethiopia' 'Fiji' 'Finland' 'France' 'Gabon' 'Gambia (the)' 'Georgia'
 'Germany' 'Ghana' 'Greece' 'Guatemala' 'Haiti' 'Holy See' 'Honduras'
 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran (Islamic Republic of)'
 'Iraq' 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan'
 'Jerusalem (

## Based on the EDA We can decide

1. Nodes:
   1. `Category`, `State`, `Region`, `Site`
2. Relationships:
   1. `BELONGS_TO` between `Site` and `Category`
   2. `LOCATED_IN` between `Site` and `State`
   3. `PART_OF` between `State` and `Region`
3. Properties:
   1. `Site`: `name`, `description`, `justification`, `year`, `longitude`, `latitude`, `area_hectares`
   2. `Category`: `name`
   3. `Region`: `name`
   4. `State`: `name`, `iso`


In [94]:
# Templates for creating the Cypher commands

from string import Template

In [95]:
# Nodes

site_template = Template(
    """\
MERGE (s:Site {
    name: "$name",
    description: "$description",
    justification: "$justification",
    year: $year,
    longitude: $longitude,
    latitude: $latitude,
    area_hectares: $area_hectares
});
    """
)

category_template = Template(
    """\
MERGE (c:Category {name: "$name"});
    """
)

region_template = Template(
    """\
MERGE (r:Region {name: "$name"});
    """
)

state_template = Template(
    """\
MERGE (s:State {name: "$name", iso: "$iso"});
    """
)

In [96]:
CYPHER_COMMANDS = []

In [97]:
unique_regions = df['region'].drop_duplicates().to_frame(name='name')
states = df[['state', 'iso']].drop_duplicates().rename(columns={'state': 'name'})
categories = df['category'].drop_duplicates().to_frame(name='name')
sites = df[['name', 'description', 'justification', 'year', 'longitude', 'latitude', 'area_hectares']]


def create_node_cypher(template, df, label):
    for i, row in df.iterrows():
        cypher_command = template.substitute(row)
        CYPHER_COMMANDS.append(cypher_command)

In [98]:
# Generate Cypher commands for each type of node
create_node_cypher(category_template, categories, 'Category')
create_node_cypher(region_template, unique_regions, 'Region')
create_node_cypher(state_template, states, 'State')
create_node_cypher(site_template, sites, 'Site')

In [99]:
# Relationships 

# templates
belongs_to_template = Template(
    """\
MATCH (s:Site {name: "$site_name"})
MATCH (c:Category {name: "$category_name"})
MERGE (s)-[:BELONGS_TO]->(c);
    """
)

located_in_template = Template(
    """\
MATCH (s:Site {name: "$site_name"})
MATCH (st:State {name: "$state_name"})
MERGE (s)-[:LOCATED_IN]->(st);
    """
)

part_of_template = Template(
    """\
MATCH (st:State {name: "$state_name"})
MATCH (r:Region {name: "$region_name"})
MERGE (st)-[:PART_OF]->(r);
    """
)

In [100]:
# Create BELONGS_TO relationships
for i, row in df.iterrows():
    command = belongs_to_template.substitute(site_name=row['name'], category_name=row['category'])
    CYPHER_COMMANDS.append(command)

In [101]:
# Create LOCATED_IN relationships
for i, row in df.iterrows():
    command = located_in_template.substitute(site_name=row['name'], state_name=row['state'])
    CYPHER_COMMANDS.append(command)

In [102]:
# Create PART_OF relationships
unique_state_region = df[['state', 'region']].drop_duplicates()
for i, row in unique_state_region.iterrows():
    command = part_of_template.substitute(state_name=row['state'], region_name=row['region'])
    CYPHER_COMMANDS.append(command)

In [103]:
# Save the Cypher commands to a file

output_path = "data/whc-sites-2018-small.cypher"
with open(output_path, 'w') as f:
    for command in CYPHER_COMMANDS:
        f.write(command)

print("DONE")

DONE


In [84]:
! pip install neo4j python-dotenv 


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [104]:
# let us exxcute the cypher commands in the neo4j database

from neo4j import GraphDatabase
import os

def run_cypher_commands(driver, file_path):
    with open(file_path, 'r') as f:
        commands = f.read()
        commands = [command.strip() for command in commands.split(';') if command.strip()]
    with driver.session() as session:
        for command in commands:
            try:
                session.run(command)
            except Exception as e:
                print(f"Error running command: {command}")
                print(e)
                input("Please Enter to continue to next command")

# clean any existing data in the database
def clean_db(driver):
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")

In [105]:
from dotenv import load_dotenv
load_dotenv()

# Create Connection

uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(uri, auth=(user, password))

In [106]:
cypher_file_path = "data/whc-sites-2018-small.cypher"
clean_db(driver)

In [107]:
# Run the Cypher commands (let's keep some log too)

import time
starting_time = time.time()
run_cypher_commands(driver, cypher_file_path)
ending_time = time.time()

print("Time taken to run the Cypher commands:", ending_time - starting_time)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Failed to parse string literal. The query must contain an even number of non-escaped quotes. (line 3, column 18 (offset: 89))
"    description: "<p>Berat and Gjirokastra are inscribed as rare examples of an architectural character typical of the Ottoman period. Located in central Albania, Berat bears witness to the coexistence of various religious and cultural communities down the centuries. It features a castle, locally known as the Kala, most of which was built in the 13th century, although its origins date back to the 4th century BC. The citadel area numbers many Byzantine churches, mainly from the 13th century, as well as several mosques built under the Ottoman era which&nbsp"
                  ^}